<a href="https://colab.research.google.com/github/GabrieleGorreri/Tokyo_Olympics_Analysis/blob/main/Tokyo_Olympics_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokyo EDA

## Set up config

In [1]:
# Install Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Set Directory folder

%cd '/content/drive/MyDrive/Colab Notebooks/Tokyo EDA'

/content/drive/MyDrive/Colab Notebooks/Tokyo EDA


In [3]:
# Check !

!pwd

/content/drive/MyDrive/Colab Notebooks/Tokyo EDA


In [4]:
# install and upgrade plotly, then restart runtime
!pip install --upgrade plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [5]:
# Making a directory
!mkdir ~/.kaggle

#Copy json file to directory
!cp kaggle.json ~/.kaggle/

# Modify permission
!chmod 600 ~/.kaggle/kaggle.json

#Download Kaggle dataset
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo

#Unzip file
!unzip 2021-olympics-in-tokyo.zip

Dataset URL: https://www.kaggle.com/datasets/arjunprasadsarkhel/2021-olympics-in-tokyo
License(s): CC-BY-SA-4.0
2021-olympics-in-tokyo.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2021-olympics-in-tokyo.zip
replace Athletes.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Athletes.xlsx           
replace Coaches.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Coaches.xlsx            
replace EntriesGender.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: EntriesGender.xlsx      
replace Medals.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Medals.xlsx             
replace Teams.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Teams.xlsx              


In [6]:
# install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=bb08a2394d1a4fe1a93176954e308cbf6dee6bfcd84b37942f3571ff512d52f4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [7]:
# import pyspark
import pyspark

In [8]:
# get access to spark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

## From xlsx to csv

In [9]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep=',', index=False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entries_gender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')

## EDA

***GOLD MEDALS***

In [11]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [12]:
# Rank by Total
medals.sort_values('Rank by Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [13]:
import plotly.express as px

fig = px.bar(data_frame= medals,
             x='Team/NOC',
             y=['Gold', 'Silver',	'Bronze'],
             color_discrete_sequence= ['Gold', 'Silver',	'Brown'],
             title = "Number of Medals by Coutry")
fig.show()

***COACHES***

In [14]:
coaches.head(5)

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [15]:
# inferSchema for putting number in proper format

spark.read.csv('Coaches.csv', header = True, inferSchema=True ).createOrReplaceTempView('Coaches')

In [16]:
spark.sql('Select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|Côte d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [17]:
pandas_coaches_query = spark.sql('Select NOC, Count(NOC) as Count from Coaches group by NOC order by Count desc')
pandas_coaches_query.show(7)

+--------------------+-----+
|                 NOC|Count|
+--------------------+-----+
|               Japan|   35|
|               Spain|   28|
|United States of ...|   28|
|           Australia|   22|
|              Canada|   16|
|               Italy|   14|
|People's Republic...|   12|
+--------------------+-----+
only showing top 7 rows



In [18]:
# Same but in Pandas
pandas_coaches_query = spark.sql('Select NOC, Count(NOC) as Count from Coaches group by NOC order by Count desc').toPandas()
pandas_coaches_query.head(7)

,NOC,Count
0,Japan,35
1,Spain,28
2,United States of America,28
3,Australia,22
4,Canada,16
5,Italy,14
6,People's Republic of China,12


In [19]:
fig = px.bar(data_frame=pandas_coaches_query,
             x='NOC',
             y='Count',
             title='Number of coaches for each country')

fig.show()

***ATHLETES***

In [20]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [21]:
spark.read.csv('Athletes.csv', header = True, inferSchema=True ).createOrReplaceTempView('Athletes')
spark.sql("select * from athletes order by NOC, Discipline").show()

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
| LAHOULOU Abdelmalik|    Algeria|          Athletics|
|TRIKI Yas

In [22]:
pandas_athletes_query = spark.sql('Select NOC, Discipline, Count(Name) as Count from Athletes group by NOC, Discipline order by Count desc')
pandas_athletes_query.show()

+--------------------+----------+-----+
|                 NOC|Discipline|Count|
+--------------------+----------+-----+
|United States of ...| Athletics|  144|
|             Germany| Athletics|   95|
|       Great Britain| Athletics|   75|
|               Italy| Athletics|   73|
|               Japan| Athletics|   70|
|              France| Athletics|   66|
|           Australia| Athletics|   65|
|              Poland| Athletics|   64|
|               Spain| Athletics|   58|
|              Canada| Athletics|   56|
|             Jamaica| Athletics|   54|
|              Brazil| Athletics|   53|
|People's Republic...| Athletics|   53|
|             Ukraine| Athletics|   48|
|         Netherlands| Athletics|   44|
|           Australia|  Football|   43|
|               Japan|  Football|   42|
|              Brazil|  Football|   41|
|United States of ...|  Swimming|   39|
|            Ethiopia| Athletics|   39|
+--------------------+----------+-----+
only showing top 20 rows



In [23]:
fig = px.bar(data_frame=pandas_athletes_query,
             x='NOC',
             y='Count',
             color = 'Discipline',
             title='Number of Athletes for each Country per Discipline')

fig.show()

***Which disciplines are the most common?***

In [24]:
# Check which disciplines are the most common
athletes['Discipline'].value_counts().head(10)

Discipline
Athletics       2068
Swimming         743
Football         567
Rowing           496
Hockey           406
Judo             373
Handball         343
Shooting         342
Sailing          336
Rugby Sevens     283
Name: count, dtype: int64

***Make some Graph --> We need to change it in Pandas df !!!***

In [25]:
#Convert pyspark df to Pandas df
athletes_df = pandas_athletes_query.toPandas()

#Filtering df
sport_df = athletes_df[athletes_df['Discipline'].isin(['Athletics', 'Swimming','Football'])]



In [26]:
#Check
print(sport_df.Discipline.unique())
print('-'*40)
print(sport_df.Discipline.describe())

['Athletics' 'Football' 'Swimming']
----------------------------------------
count           400
unique            3
top       Athletics
freq            195
Name: Discipline, dtype: object


In [27]:
#Plot

fig = px.bar(data_frame = sport_df,
             x='NOC',
             y='Count',
             color = 'Discipline',
             title='Number of Athletes for each Country per Discipline')

fig.show()

***Another way to filter the dataset is:***



In [28]:
athletes_df = athletes_df.loc[athletes_df['Discipline'].isin(['Athletics', 'Swimming','Football'])]
athletes_df.Discipline.value_counts()

Discipline
Athletics    195
Swimming     181
Football      24
Name: count, dtype: int64

***GENDER***

In [29]:
entries_gender.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [30]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Here we are setting the pie charts
specs = [[{'type':'domain'}, {'type':'domain'}]] * 23
specs

[[{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}]]

In [31]:
#Specifing the number of rows and columns
fig = make_subplots(rows=23, cols=2,
                    subplot_titles=entries_gender['Discipline'], #the Discipline column, as pandas list
                    specs=specs)

sub_figs = []
for index, row in entries_gender.iterrows():
  sub_fig = go.Pie(labels=['Female','Male'], values=[row['Female'],
                                                     row['Male']])
  sub_figs.append(sub_fig)


k = 0
for i in range(1, 24): # 1 to 23 (last number in range is not considered)
  for j in range(1, 3): # 1 to 2
    fig.add_trace(sub_figs[k], i, j)
    k += 1

fig.update_layout(showlegend=False, height=10000, width=800, title_text="Distribution of Gender amongst each game")
fig.update_traces(textposition='inside', textinfo='label+percent', hoverinfo='label+value+percent')

fig = go.Figure(fig)
fig.show()